## Asif Arman Rahman
***
***
***

In [2]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn_pandas import CategoricalImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


%matplotlib inline
warnings.filterwarnings('ignore')

ImportError: cannot import name 'CategoricalImputer' from 'sklearn_pandas' (C:\Users\Asif\miniconda3\envs\main\lib\site-packages\sklearn_pandas\__init__.py)

In [ ]:
train_df = pd.read_csv('data/train.csv')
test_df =  pd.read_csv('data/test.csv')
pd.options.display.max_columns = None

In [ ]:
train_df.info()

In [ ]:
print ("Size of train data : {}" .format(train_df.shape))

print ("Size of test data : {}" .format(test_df.shape))

In [ ]:
#Save the 'Id' column
train_ID = train_df['Id']
test_ID = test_df['Id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train_df.drop("Id", axis = 1, inplace = True)
test_df.drop("Id", axis = 1, inplace = True)

## Q1 (A)

### Missing data and Categorical data Handle
***

In [ ]:
# Selecting only the numeric columns
train = train_df.select_dtypes(['int64', 'float64'])
test = test_df.select_dtypes(['int64', 'float64'])

# Dropping the insignificant features
train = train.drop(['GarageYrBlt', 'TotRmsAbvGrd', '1stFlrSF', 'GarageCars'], axis=1)
test = test.drop(['GarageYrBlt', 'TotRmsAbvGrd', '1stFlrSF', 'GarageCars'], axis=1)

In [ ]:
train_na = (train.isnull().sum() / len(train)) * 100
train_na = train_na.drop(train_na[train_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :train_na})


f, ax = plt.subplots(figsize=(4, 4))
plt.xticks(rotation='90')
sns.barplot(x=train_na.index, y=train_na)
plt.xlabel('Features', fontsize=8)
plt.ylabel('Percent of missing values', fontsize=8)
plt.title('Percent missing data by feature', fontsize=10)
plt.show()

In [ ]:
test_na = (test.isnull().sum() / len(test)) * 100
test_na = test_na.drop(test_na[test_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :test_na})


f, ax = plt.subplots(figsize=(4, 4))
plt.xticks(rotation='90')
sns.barplot(x=test_na.index, y=test_na)
plt.xlabel('Features', fontsize=8)
plt.ylabel('Percent of missing values', fontsize=8)
plt.title('Percent missing data by feature', fontsize=10)
plt.show()

In [ ]:
# The unique value count in each feature
cat_count = train.apply(lambda x: x.value_counts().shape[0]).sort_values()

In [ ]:
# Categorical columns in numerical columns
cats_in_nums = cat_count.loc[cat_count < 50].index

# Converting the type of filtered numerical columns to categorical
train.loc[:, cats_in_nums] = train.loc[:, cats_in_nums].astype('object')
test.loc[:, cats_in_nums] = test.loc[:, cats_in_nums].astype('object')

train.info()

In [ ]:
# Adding Neighborhood feature to the filtered datasets
train['Neighborhood'] = train_df['Neighborhood']
test['Neighborhood'] = test_df['Neighborhood']

corrTrain = train

# Extracting the SalePrice from training data
y_train = np.log(train['SalePrice'].values)
train = train.drop(['SalePrice'], axis=1)

In [ ]:
# Imputing the missing numerical values with the median value as the features are not uniformly distributed

imputer = SimpleImputer(strategy='median')
train_num = imputer.fit_transform(train.select_dtypes(['int64', 'float64']))
test_num = imputer.transform(test.select_dtypes(['int64', 'float64']))

In [ ]:
# Imputing the missing categorical value with the most frequent value

cat_columns = train.select_dtypes(['object']).columns
cat_imputer = CategoricalImputer()
train_cat = cat_imputer.fit_transform(train.loc[:, cat_columns].values)
test_cat = cat_imputer.transform(test.loc[:, cat_columns].values)

In [ ]:
# One-hot encoding the categorical columns

one_hot_encoder = OneHotEncoder()
combined_cat_data = np.vstack([train_cat, test_cat])
combined_cat = one_hot_encoder.fit_transform(combined_cat_data).todense()

train_cat = combined_cat[:train.shape[0]]
test_cat = combined_cat[train.shape[0]:]

***
## Q1 (B)

### Correlation matrix (in terms of target variable/feature)
***

In [ ]:
# Correlation Matrix

plt.figure(figsize=(30,30))
plt.title('Pearson Correlation of Features', y=1.05, size=70)
sns.heatmap(corrTrain.corr(), cmap='Spectral', linewidths=1.5, square=True, annot=True)
plt.show()

***
## Q1 (C)

### Feature Scaling the numeric columns

***

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Creating a scaler for input features
scaler = MinMaxScaler()

# Transforming the input features of both train and test
train_num = scaler.fit_transform(train_num)
test_num = scaler.transform(test_num)

In [ ]:
X_train = np.hstack((train_num, train_cat))
X_test = np.hstack((test_num, test_cat))

***
## Q2 (A)

### Use Support Vector Machine to predict Housing Price
***

In [ ]:
def rmse(model):
    error = np.sqrt(abs(cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')))
    
    return error

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

svr = SVR(kernel='linear')
print(rmse(svr).mean(), rmse(svr).std())

***
## Q2 (B)

### Decision Tree with maximum depth restriction
****

In [ ]:
tree_reg = DecisionTreeRegressor(random_state=42, max_depth=3)

print(rmse(tree_reg).mean(), rmse(tree_reg).std())

### Decision Tree with no maximum depth restriction

In [ ]:
tree_reg = DecisionTreeRegressor(random_state=42, max_depth=None)

print(rmse(tree_reg).mean(), rmse(tree_reg).std())

***
## Q2 (C)

### Random Forest with various n_estimators count
***

In [ ]:
array =  [10, 100, 1000]

for i in array:
    forest_reg = RandomForestRegressor(n_estimators=i)
    
    print('For n_estimator: ', i, ' ', rmse(tree_reg).mean(), rmse(tree_reg).std())

***
## Q3

### Perform PCA and then apply random forest algorithm
***

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=120)
pca.fit(X_train)

x_pca = pca.transform(X_train)
x_test_pca = pca.transform(X_test)

In [ ]:
forest_reg = RandomForestRegressor()

error = np.sqrt(abs(cross_val_score(forest_reg, x_pca, y_train, cv=5, scoring='neg_mean_squared_error')))

print(error.mean(),error.std())


As it can be seen that applying PCA does improve RandomForestRegressor score.

***
# Submission
***

In [ ]:
forest_reg.fit(x_pca,y_train)
house = forest_reg.predict(x_test_pca)

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = house
sub.to_csv('submission.csv',index=False)